## Read all 12 txt files from US_VitalStatistics.zip

In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.set_option("mode.copy_on_write", True)

# list of 12 names for txt files from US_VitalStatistics.zip
file_names = [
    "Underlying Cause of Death, 2003",
    "Underlying Cause of Death, 2004",
    "Underlying Cause of Death, 2005",
    "Underlying Cause of Death, 2006",
    "Underlying Cause of Death, 2007",
    "Underlying Cause of Death, 2008",
    "Underlying Cause of Death, 2009",
    "Underlying Cause of Death, 2010",
    "Underlying Cause of Death, 2011",
    "Underlying Cause of Death, 2012",
    "Underlying Cause of Death, 2013",
    "Underlying Cause of Death, 2014",
    "Underlying Cause of Death, 2015",
]

# create a new list to contain combined data
data_frames = []

# states to select
selected_states = [", TX", ", FL", ", WA"]

for name in file_names:
    # construct the file path，you could change your path
    path = f"/workspaces/opioid-2023-group-8-final-opioid/10_Code/US_VitalStatistics/{name}.txt"

    # read txt file
    state_death = pd.read_csv(path, sep="\t")

    # select only the rows for TX, FL, and WA
    selected_data = state_death.loc[
        state_death["County"].notna()
        & state_death["County"].str.contains("|".join(selected_states)),
        :,
    ]

    # append to the list of dataframe
    data_frames.append(selected_data)

# concatenate all dataframe in the list
combined_df = pd.concat(data_frames, ignore_index=True)

In [2]:
combined_df.sample(5)

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
3501,NaN,"Fannin County, TX",48147.0,2009.0,2009.0,All other non-drug and non-alcohol causes,O9,439.0
6412,NaN,"Lubbock County, TX",48303.0,2014.0,2014.0,All other non-drug and non-alcohol causes,O9,2332.0
1748,NaN,"Sumter County, FL",12119.0,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,870.0
1498,NaN,"Val Verde County, TX",48465.0,2005.0,2005.0,All other non-drug and non-alcohol causes,O9,300.0
6821,NaN,"Brazoria County, TX",48039.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,2119


## Check for Missing Values



In [3]:
missing_values = combined_df.isna().sum()
print("Missing values in the dataset:")
missing_values

Missing values in the dataset:


Notes                              7216
County                                0
County Code                           0
Year                                  0
Year Code                             0
Drug/Alcohol Induced Cause            0
Drug/Alcohol Induced Cause Code       0
Deaths                                0
dtype: int64

## Convert "Year" to Categorical Data Type

In [4]:
combined_df["Year"].dtype

dtype('float64')

In [5]:
# Convert 'Year' from float to categorical
combined_df["Year"] = combined_df["Year"].astype("category")

# Check the new data type
combined_df["Year"].dtype

CategoricalDtype(categories=[2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0,
                  2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0],
, ordered=False, categories_dtype=float64)

## Filter Drug Related Deaths Only

In [6]:
combined_df["Drug/Alcohol Induced Cause"].value_counts()

Drug/Alcohol Induced Cause
All other non-drug and non-alcohol causes             4586
Drug poisonings (overdose) Unintentional (X40-X44)    1078
All other alcohol-induced causes                      1049
Drug poisonings (overdose) Suicide (X60-X64)           322
All other drug-induced causes                           91
Drug poisonings (overdose) Undetermined (Y10-Y14)       48
Alcohol poisonings (overdose) (X45, X65, Y15)           42
Name: count, dtype: int64

In [21]:
# Filter for drug-related deaths
drug_related_causes = [
    "Drug poisonings (overdose) Unintentional (X40-X44)",
    "Drug poisonings (overdose) Suicide (X60-X64)",
    "All other drug-induced causes",
    "Drug poisonings (overdose) Undetermined (Y10-Y14)",
]

# Filter for drug-related deaths
drug_related_df = combined_df[
    combined_df["Drug/Alcohol Induced Cause"].isin(drug_related_causes)
]
drug_related_df.head()

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State
0,NaN,"Alachua County, FL",12001.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,FL
4,NaN,"Bay County, FL",12005.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,21.0,FL
8,NaN,"Brevard County, FL",12009.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,83.0,FL
9,NaN,"Brevard County, FL",12009.0,2003.0,2003.0,Drug poisonings (overdose) Suicide (X60-X64),D2,14.0,FL
12,NaN,"Broward County, FL",12011.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,170.0,FL


## Create a "State" Colunm 

In [22]:
# Define a function to extract the state abbreviation
def extract_state(county_name):
    # Split the string by comma and take the last part as the state
    parts = county_name.split(", ")
    if len(parts) > 1:
        return parts[-1].strip()
    return None


# Apply the function to create a new 'State' column
drug_related_df["State"] = drug_related_df["County"].apply(extract_state)
drug_related_df

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State
0,NaN,"Alachua County, FL",12001.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,FL
4,NaN,"Bay County, FL",12005.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,21.0,FL
8,NaN,"Brevard County, FL",12009.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,83.0,FL
9,NaN,"Brevard County, FL",12009.0,2003.0,2003.0,Drug poisonings (overdose) Suicide (X60-X64),D2,14.0,FL
12,NaN,"Broward County, FL",12011.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,170.0,FL
...,...,...,...,...,...,...,...,...,...
7198,NaN,"Spokane County, WA",53063.0,2015.0,2015.0,Drug poisonings (overdose) Unintentional (X40-...,D1,74,WA
7199,NaN,"Spokane County, WA",53063.0,2015.0,2015.0,Drug poisonings (overdose) Suicide (X60-X64),D2,18,WA
7203,NaN,"Thurston County, WA",53067.0,2015.0,2015.0,Drug poisonings (overdose) Unintentional (X40-...,D1,26,WA
7209,NaN,"Whatcom County, WA",53073.0,2015.0,2015.0,Drug poisonings (overdose) Unintentional (X40-...,D1,23,WA


## Convert "Deaths" to Numeric

In [32]:
drug_related_df["Deaths"].dtype

dtype('O')

In [36]:
drug_related_df["Deaths"] = pd.to_numeric(drug_related_df["Deaths"], errors="coerce")
drug_related_df["Deaths"].dtype

float64


## Create a new df with Year, County, State, Drug-related Deaths

The issue I met here is that the "Deaths" column displaying zeros after the groupby operation. To effectively address this, we'll refine our approach to ensure that the data in the "Deaths" column is correctly interpreted as numeric and properly aggregated. This involves converting the "Deaths" column to a numeric format, accurately extracting state abbreviations, and then performing a groupby operation that groups by "Year", "County", and "State", followed by summing the deaths. By carefully following these steps, we aim to produce a DataFrame that accurately reflects drug-related deaths in each county and state for each year.

In [48]:
# Check unique values in each grouping column
print(drug_related_df["Year"].unique())
print(drug_related_df["County"].unique())
print(drug_related_df["County Code"].unique())
print(drug_related_df["State"].unique())

[2003.0, 2004.0, 2005.0, 2006.0, 2007.0, ..., 2011.0, 2012.0, 2013.0, 2014.0, 2015.0]
Length: 13
Categories (13, float64): [2003.0, 2004.0, 2005.0, 2006.0, ..., 2012.0, 2013.0, 2014.0, 2015.0]
['Alachua County, FL' 'Bay County, FL' 'Brevard County, FL'
 'Broward County, FL' 'Citrus County, FL' 'Clay County, FL'
 'Collier County, FL' 'Duval County, FL' 'Escambia County, FL'
 'Hernando County, FL' 'Highlands County, FL' 'Hillsborough County, FL'
 'Indian River County, FL' 'Lake County, FL' 'Lee County, FL'
 'Leon County, FL' 'Manatee County, FL' 'Marion County, FL'
 'Martin County, FL' 'Miami-Dade County, FL' 'Monroe County, FL'
 'Okaloosa County, FL' 'Orange County, FL' 'Osceola County, FL'
 'Palm Beach County, FL' 'Pasco County, FL' 'Pinellas County, FL'
 'Polk County, FL' 'St. Lucie County, FL' 'Santa Rosa County, FL'
 'Sarasota County, FL' 'Seminole County, FL' 'Volusia County, FL'
 'Bexar County, TX' 'Brazoria County, TX' 'Collin County, TX'
 'Comal County, TX' 'Dallas County, TX' '

In [57]:
# Inspect the data before grouping
print(drug_related_df.sample(5))

     Notes                 County  County Code    Year  Year Code  \
1712   NaN  Palm Beach County, FL      12099.0  2006.0     2006.0   
3264   NaN     Broward County, FL      12011.0  2009.0     2009.0   
3934   NaN   St. Lucie County, FL      12111.0  2010.0     2010.0   
159    NaN       Bexar County, TX      48029.0  2003.0     2003.0   
4028   NaN      Dallas County, TX      48113.0  2010.0     2010.0   

                             Drug/Alcohol Induced Cause  \
1712  Drug poisonings (overdose) Undetermined (Y10-Y14)   
3264  Drug poisonings (overdose) Undetermined (Y10-Y14)   
3934  Drug poisonings (overdose) Unintentional (X40-...   
159        Drug poisonings (overdose) Suicide (X60-X64)   
4028       Drug poisonings (overdose) Suicide (X60-X64)   

     Drug/Alcohol Induced Cause Code  Deaths State Correct_State  
1712                              D4    21.0    FL            FL  
3264                              D4    16.0    FL            FL  
3934                         

In [51]:
# Simplified grouping: Initially group by fewer columns
simplified_grouped_df = (
    drug_related_df.groupby(["Year", "County"])["Deaths"].sum().reset_index()
)

# Check the simplified grouped DataFrame
print(simplified_grouped_df.sample(5))

        Year                County  Deaths
898   2010.0     Kitsap County, WA    19.0
1415  2014.0   Sarasota County, FL    54.0
999   2011.0  Guadalupe County, TX    11.0
62    2003.0       Levy County, FL     0.0
376   2006.0    Clallam County, WA    15.0


This suggests that the problem probably is not with the "State" column but potentially with the "County Code" or how the counties are labeled in the dataset. There might be discrepancies in the "County Code" values for the same "County", leading to the splitting of data and resulting in zero values.

In [53]:
# Group by Year, County, and County Code, and sum the deaths
grouped_df_with_code = (
    drug_related_df.groupby(["Year", "County", "County Code"])["Deaths"]
    .sum()
    .reset_index()
)

# Check the grouped DataFrame
print(grouped_df_with_code.sample(5))


          Year               County  County Code  Deaths
382     2003.0   Bastrop County, TX      12085.0     0.0
136587  2012.0    Kitsap County, WA      12093.0     0.0
67359   2007.0    Orange County, TX      12117.0     0.0
118724  2011.0   El Paso County, TX      48007.0     0.0
53759   2006.0  Pinellas County, FL      53077.0     0.0


In [55]:
# Group by Year, County, and State, and sum the deaths
grouped_df_without_code = (
    drug_related_df.groupby(["Year", "County", "State"])["Deaths"].sum().reset_index()
)

# Check the grouped DataFrame
print(grouped_df_without_code.sample(5))

        Year              County State  Deaths
4201  2014.0   Orange County, FL    TX     0.0
351   2003.0  Wichita County, TX    FL     0.0
963   2005.0   Orange County, TX    FL     0.0
3637  2013.0  Cameron County, TX    TX     0.0
3047  2011.0  Kaufman County, TX    WA     0.0


In the process of addressing the issue with the "Deaths" column displaying zeros, I made a decision to group the data by "Year", "County", and "State". This approach means I'm not including the "County Code" in the grouping.

When I perform a groupby operation in pandas, each group is defined by the unique combinations of values in the specified columns. Since I am grouping by "Year", "County", and "State", the "County Code" is not part of this unique combination, and thus not included in the final grouped DataFrame.

However, if the "County Code" is important for the future analysis, we could potentially include it in the grouping. This would result in groups defined by the unique combinations of "Year", "County", "County Code", and "State". Just be aware that if the "County Code" varies for the same "County" and "State" across different years, this would create more granular groups, potentially affecting the aggregation of deaths.

In [60]:
# Define a function to extract the correct state abbreviation
def extract_correct_state(county_name):
    # Extract the state abbreviation from the county name
    parts = county_name.split(", ")
    if len(parts) > 1:
        return parts[-1].strip()
    return None


# Apply the function to create a new 'Correct_State' column
drug_related_df["Correct_State"] = drug_related_df["County"].apply(
    extract_correct_state
)

# Group by Year, County, and Correct_State, and sum the deaths
grouped_correct_df = (
    drug_related_df.groupby(["Year", "County", "Correct_State"])["Deaths"]
    .sum()
    .reset_index()
)

# Check the grouped DataFrame
grouped_correct_df.sample(5)

,Year,County,Correct_State,Deaths
615,2004.0,"Pasco County, FL",FL,83.0
2839,2010.0,"Taylor County, TX",TX,12.0
876,2005.0,"Island County, WA",FL,0.0
3927,2013.0,"Travis County, TX",FL,0.0
1404,2006.0,"Tom Green County, TX",FL,0.0


## Washington

In [61]:
# Filter for rows where both 'County' and 'State' columns indicate Washington State
wa_data = grouped_correct_df[
    (grouped_correct_df["County"].str.contains(", WA"))
    & (grouped_correct_df["Correct_State"] == "WA")
]
wa_data.sample(5)

,Year,County,Correct_State,Deaths
551,2004.0,"Lewis County, WA",WA,0.0
2564,2010.0,"Chelan County, WA",WA,11.0
3710,2013.0,"Grays Harbor County, WA",WA,12.0
1547,2007.0,"Grant County, WA",WA,11.0
3239,2011.0,"Yakima County, WA",WA,14.0


## Taxas

In [62]:
# Filter for Texas
tx_data = grouped_correct_df[
    (grouped_correct_df["County"].str.contains(", TX"))
    & (grouped_correct_df["Correct_State"] == "TX")
]
tx_data.sample(5)

,Year,County,Correct_State,Deaths
1606,2007.0,"Kaufman County, TX",TX,0.0
2488,2009.0,"Travis County, TX",TX,115.0
4546,2015.0,"Nacogdoches County, TX",TX,15.0
115,2003.0,"Gregg County, TX",TX,0.0
724,2005.0,"Anderson County, TX",TX,0.0


## Florida

In [63]:
# Filter for Florida
fl_data = grouped_correct_df[
    (grouped_correct_df["County"].str.contains(", FL"))
    & (grouped_correct_df["Correct_State"] == "FL")
]
fl_data.sample(5)

,Year,County,Correct_State,Deaths
2937,2011.0,"Collier County, FL",FL,41.0
4113,2014.0,"Indian River County, FL",FL,22.0
3216,2011.0,"Volusia County, FL",FL,89.0
4092,2014.0,"Hernando County, FL",FL,25.0
4200,2014.0,"Orange County, FL",FL,177.0


## Dealing with Missing Values
（wating for the population dataset ready)

Suggestion from Nick in our assignement:

In light of that, I think there are a couple choices:

Calculate the mortality rate for counties, then fill in missing mortality rates with the average mortality rate (not count) for the county's state and year. But to some extent this basically just becomes "making up data" given how many counties will have no data at any point during this period until you do these imputations. 
Limit your analysis to counties that are big enough that the "only report where 10 or more deaths occur" rule just never comes into play. This is what's called a "scope condition": you're basically limiting your analysis to the study of big counties. This is different from dropping counties without data because when you do that, you are systematically more likely to drop counties with less of an opioid problem (in the social sciences we call this "selecting on the dependent variable"). 

In [ ]:
# Jiayi has already load this dataset, I load it by mistake

# import pandas as pd
# import warnings

# warnings.filterwarnings("ignore")
# pd.set_option("mode.copy_on_write", True)

# # summary of mortality for drug and non-drug-related causes for every US county from 2003-2015
# data_arcos = "/Users/castnut/Desktop/720_Practical_Data_Science/final_project/arcos_all_washpost.tsv"

# chunk_size = 1000
# chunks = []

# for chunk in pd.read_csv(
#     data_arcos, sep="\t", chunksize=chunk_size, skiprows=lambda i: i % 10 != 0
# ):
#     chunks.append(chunk)

# arcos_combined = pd.concat(chunks, ignore_index=True)

## 1. What problem are you (or your stakeholder) trying to address?
The problem we are addressing is the epidemic of opioid misuse and overdose in the United States. The situation has worsened due to doctors prescribing too many opioids, causing more people to become addicted and leading to more deaths from overdoses. The stakeholders in this scenario could be policymakers, healthcare providers, and public health officials who are concerned with the impact of opioid abuse on individuals and communities. The goal is to evaluate the effectiveness of policy interventions designed to limit the over-prescription of opioids and to assess their impact on both opioid drug prescriptions and mortality from drug overdoses.

## 2. What specific question are you seeking to answer with this project?
The specific question we are seeking to answer is: 

 **How do policies aimed at regulating the prescription of opioid medications influence the amount of opioids dispensed and the number of fatalities due to drug overdoses?**

This question aims to causally link policy interventions with observable outcomes in the realm of public health and drug regulation. By understanding the effects of these policies, stakeholders can make informed decisions about how to best address the opioid misuse.

## 3. What is your hypothesized answer to your question?

**Strict opioid prescription regulation policies will lead to a decrease in the volume of opioids prescribed.**

This could be due to tighter controls on prescription practices, better monitoring of opioid distribution, and increased awareness and education about the risks of opioid misuse. However, while a reduction in opioid prescriptions might be expected, the impact on drug overdose deaths is more complex. It's possible that, in the short term, overdose deaths might increase as individuals who are already addicted turn to illegal and more dangerous substances like heroin or fentanyl. In the long term, however, if the policies are effective in preventing new cases of addiction, we might expect to see a gradual decrease in overdose deaths.


Further dicussion:
1. may need additional dataset as the requirement asked.


year(categorical), county, county-code, state, sum(drug-related-deaths)

a 10 drug 2010
a 10 drug2 2010

2010 a 20